In [ ]:
from importlib import reload
import ipynb.fs.full.notebook_viewer
reload(ipynb.fs.full.notebook_viewer)
from ipynb.fs.full.notebook_viewer import Viewer, Manager, root, fname2id, id2fname, split_dump, search_dump, Filter, prepare, tqdm_load, id2, load_wimage
import pickle
from pickle import Unpickler
from pprint import pprint
from dotdict import dotdict
from functools import reduce

from typing import *
import matplotlib.pyplot as plt
import random
import PIL
from pathlib import Path
import json
from collections import defaultdict
import subprocess
from subprocess import PIPE
from tqdm.notebook import tqdm
from time import time
from functools import lru_cache
import numpy as np
from PIL import Image, ImageDraw
import imagesize
from math import atan2, degrees

In [ ]:
face_one = set((root/"danbooru2020/dump_face_one/0").read_text().strip().split())
instance_one = set((root/"danbooru2020/dump_instance_one/0").read_text().strip().split())

In [ ]:
fl = dict()
fl["faceinstance"] = (face_one & instance_one) - {'3634238','3634244'}

In [ ]:
samesize = defaultdict(list)
for _id in tqdm(fl["faceinstance"]):
    fname = id2.originalwhite(_id)
     m = tuple(map(int, map(ids[_id].get, ['image_width', 'image_height'])))
#    b = imagesize.get(fname)
#    assert a == b
    samesize[ m].append(_id)

In [ ]:
split_dump(["\t".join(v) for v in samesize.values()        len(v) > 10 ], "samesize"  , N= 1)
split_dump(["\t".join(v) for v in samesize.values() if 1 < len(v) <= 10], "samesize10", N=-1)

In [ ]:
results = list((root/"danbooru2020/samesize").glob("*"))

In [ ]:
THR = 40
ok = list()
for i, fname in enumerate(tqdm(results)):
    with fname.open("r") as f:
        for line in f:
            m, f1name, f2name = line.strip().split()
            m = float(m)
            if THR < m:
                break
            else:
                ok.append((m, f1name, f2name))
ok.sort()

In [ ]:
def append_option(arr):
    m, f1name, f2name = arr
    _id = f"{fname2id(f1name)}_{fname2id(f2name)}"
    dname = id2fname(_id, prefix="issame", bucket=lambda _:".")
    return f"+append {f1name} {f2name} {dname}"
(root/"danbooru2020/issame").mkdir(exist_ok=True)

In [ ]:
split_dump(list(map(append_option, [a for a in ok if  m[0] < 30])), "option_issame")

In [ ]:
len(["\t".join(v) for v in samesize.values() if 1 < len(v)])

In [ ]:
search_dump("3096978", "samesize") # 10
search_dump("4010812", "samesize") # 35

In [ ]:
from itertools import combinations
split_dump(
    ["\t".join(c) for c in combinations((root/"danbooru2020/dump_samesize/10").read_text().strip().split(), 2)] + 
    ["\t".join(c) for c in combinations((root/"danbooru2020/dump_samesize/35").read_text().strip().split(), 2)]
,"samesize100")

In [ ]:
split_dump([f"-resize 512x512 -background white -gravity center -extent 512x512 {fname}" for fname in fnames], "samesize_resize")

In [ ]:
fnames = [id2fname(f"{fname2id(f1)}_{fname2id(f2)}", prefix="issame", bucket=lambda _:".") for m,f1,f2 in ok if m < 30]

In [ ]:
for i, fname in enumerate(tqdm(fnames)):
    _ = load_wimage(fname)

In [ ]:
reduce_duplicates = Viewer(fnames, key = "nosort", name ="reduce_duplicates", N=12*6-1)
reduce_duplicates

In [ ]:
reduce_duplicates.bag |= set(reduce_duplicates.fnames[:(12*6-1)*150])